In [287]:
import pandas as pd
import tweepy
import re
import numpy as np
import os

In [288]:
df_tweets = pd.read_csv("twitter_human_bots_dataset.csv")
df_celeb = pd.read_csv("Top-1000-Celebrity-Twitter-Accounts.csv")

In [22]:
df_tweets

,id,account_type
0,787405734442958848,bot
1,796216118331310080,human
2,875949740503859204,human
3,756119643622735875,human
4,464781334,human
...,...,...
37433,63963107,human
37434,1064042478,human
37435,1089732602,human
37436,815529979,human


In [21]:
# name, username, created_at, protected, verified, location, description, 
# public_metrics (followers_count, following_count, tweets_count)
# id, tweets, account_type
# CHECK
# favourites_count, geo_enabled, statuses_count
# average_tweets_per_day, account_age_days

def get_twitter_dictionary(id_list, account_type_dict, client):
    twitter_data = {'id':[], 'name':[], 'username':[], 'created_at':[], 
                    'protected':[], 'verified':[], 'location':[], 
                    'description':[], 'followers_count':[], 'following_count':[],
                    'tweet_count':[], 'tweets':[], 'account_type':[]}

    user_info = client.get_users(ids = id_list, user_fields=["created_at", "description", "entities", "id", "location", "name", "profile_image_url", "protected", "public_metrics", "url", "username", "verified", "withheld"])
    user_dict = dict()
    info_list = user_info[0]
    for each_user in info_list:
        data = each_user.data
        user_dict[data['id']] = data

    for item in user_dict.items():
        each_user_id = item[0]
        each_user_account_type = account_type_dict[each_user_id]
        each_user_info = item[1]
        tweets = client.get_users_tweets(each_user_id, max_results = 5)

        if not tweets[0]:
            continue
        for tweet in tweets[0]:
            txt = tweet.text

            twitter_data['id'].append(each_user_id)
            twitter_data['tweets'].append(txt)
            twitter_data['account_type'].append(each_user_account_type)
            each_dictionary = user_dict[each_user_id]

            if 'name' in each_dictionary:
                twitter_data['name'].append(each_dictionary['name'])
            else:
                twitter_data['name'].append('')

            if 'username' in each_dictionary:
                twitter_data['username'].append(each_dictionary['username'])
            else:
                twitter_data['username'].append('')

            if 'created_at' in each_dictionary:
                twitter_data['created_at'].append(each_dictionary['created_at'])
            else:
                twitter_data['created_at'].append('')

            if 'protected' in each_dictionary:
                twitter_data['protected'].append(each_dictionary['protected'])
            else:
                twitter_data['protected'].append('')

            if 'verified' in each_dictionary:
                twitter_data['verified'].append(each_dictionary['verified'])
            else:
                twitter_data['verified'].append('')

            if 'location' in each_dictionary:
                twitter_data['location'].append(each_dictionary['location'])
            else:
                twitter_data['location'].append('')

            if 'description' in each_dictionary:
                twitter_data['description'].append(each_dictionary['description'])
            else:
                twitter_data['description'].append('')

            if 'public_metrics' in each_dictionary:
                each_pub_metric = each_dictionary['public_metrics']

                if 'followers_count' in each_pub_metric:
                    twitter_data['followers_count'].append(each_pub_metric['followers_count'])
                else:
                    twitter_data['followers_count'].append('')

                if 'following_count' in each_pub_metric:
                    twitter_data['following_count'].append(each_pub_metric['following_count'])
                else:
                    twitter_data['following_count'].append('')

                if 'tweet_count' in each_pub_metric:
                    twitter_data['tweet_count'].append(each_pub_metric['tweet_count'])
                else:
                    twitter_data['tweet_count'].append('')
            else:
                twitter_data['followers_count'].append('')
                twitter_data['following_count'].append('')
                twitter_data['tweet_count'].append('')
        
    return twitter_data

In [33]:
def data_manipulation(combined_twitter_data, pattern):
    combined_twitter_data['url']= combined_twitter_data["tweets"].str.extract(pattern, expand=True)

    combined_twitter_data.loc[(combined_twitter_data['url'] != np.nan) & (combined_twitter_data["tweets"].str.startswith("RT @")), 
                              'is_quoted_retweets'] = 1

    combined_twitter_data.loc[(combined_twitter_data['is_quoted_retweets'] != 1, 'is_quoted_retweets')] = 0

    combined_twitter_data['is_quoted_retweets'] = combined_twitter_data['is_quoted_retweets'].astype(int)

    combined_twitter_data = combined_twitter_data.reset_index(inplace = False, drop = True)
    
    return combined_twitter_data

In [142]:
# name, username, created_at, protected, verified, location, description, 
# public_metrics (followers_count, following_count, tweets_count)
# id, tweets, account_type
# CHECK
# favourites_count, geo_enabled, statuses_count
# average_tweets_per_day, account_age_days

def get_features_dictionary(id_list, account_type_dict, client):
    twitter_data = {'id':[], 'name':[], 'username':[], 'created_at':[], 
                    'protected':[], 'verified':[], 'location':[], 
                    'description':[], 'followers_count':[], 'following_count':[],
                    'tweet_count':[], 'account_type':[]}

    user_info = client.get_users(ids = id_list, user_fields=["created_at", "description", "entities", "id", "location", "name", "profile_image_url", "protected", "public_metrics", "url", "username", "verified", "withheld"])
    user_dict = dict()
    info_list = user_info[0]
    if info_list == None:
        return twitter_data
    
    for each_user in info_list:
        data = each_user.data
        user_dict[data['id']] = data

    for item in user_dict.items():
        each_user_id = item[0]
        each_user_account_type = account_type_dict[each_user_id]
        each_user_info = item[1]
        tweets = client.get_users_tweets(each_user_id, max_results = 5)
        twitter_data['id'].append(each_user_id)
        twitter_data['account_type'].append(each_user_account_type)
        each_dictionary = user_dict[each_user_id]
        
        if 'name' in each_dictionary:
                twitter_data['name'].append(each_dictionary['name'])
        else:
            twitter_data['name'].append('')

        if 'username' in each_dictionary:
            twitter_data['username'].append(each_dictionary['username'])
        else:
            twitter_data['username'].append('')

        if 'created_at' in each_dictionary:
            twitter_data['created_at'].append(each_dictionary['created_at'])
        else:
            twitter_data['created_at'].append('')

        if 'protected' in each_dictionary:
            twitter_data['protected'].append(each_dictionary['protected'])
        else:
            twitter_data['protected'].append('')

        if 'verified' in each_dictionary:
            twitter_data['verified'].append(each_dictionary['verified'])
        else:
            twitter_data['verified'].append('')

        if 'location' in each_dictionary:
            twitter_data['location'].append(each_dictionary['location'])
        else:
            twitter_data['location'].append('')

        if 'description' in each_dictionary:
            twitter_data['description'].append(each_dictionary['description'])
        else:
            twitter_data['description'].append('')

        if 'public_metrics' in each_dictionary:
            each_pub_metric = each_dictionary['public_metrics']

            if 'followers_count' in each_pub_metric:
                twitter_data['followers_count'].append(each_pub_metric['followers_count'])
            else:
                twitter_data['followers_count'].append('')

            if 'following_count' in each_pub_metric:
                twitter_data['following_count'].append(each_pub_metric['following_count'])
            else:
                twitter_data['following_count'].append('')

            if 'tweet_count' in each_pub_metric:
                twitter_data['tweet_count'].append(each_pub_metric['tweet_count'])
            else:
                twitter_data['tweet_count'].append('')
        else:
            twitter_data['followers_count'].append('')
            twitter_data['following_count'].append('')
            twitter_data['tweet_count'].append('')

    return twitter_data

In [ ]:
start_index = 9
pattern = r'(https?:\/\/(?:www\.)?[-a-zA-Z0-9@:%._+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}[-a-zA-Z0-9()@:%_+.~#?&/=]*)' 

for each_round in range(16):
    token = bearer_tokens[each_round%3]
    client = tweepy.Client(bearer_token= token)
    print(token)
    
    twitter_data_list = []
    for i in range(15):
        start = i * 100
        end = start + 100
        df_tweet_subset = df_tweets[start: end]
        acc_type_dict = dict(zip(df_tweet_subset["id"].astype(str), df_tweet_subset["account_type"]))
        each_id_list = list(df_tweet_subset["id"])
        print(str(start) + " : " + str(end)) 
        twitter_data = get_twitter_dictionary(each_id_list, acc_type_dict, client)
        tweets_df = pd.DataFrame(twitter_data)
        twitter_data_list.append(tweets_df)
    
    start_index += 1
    
    combined_twitter_data = pd.concat(twitter_data_list)
    
    combined_twitter_data = data_manipulation(combined_twitter_data, pattern)
    
    combined_twitter_data.to_csv(f'combined/combined_twitter_data{each_round+10}.csv')
    
    print("round" + str(each_round+10))

In [214]:
start_index = 9
pattern = r'(https?:\/\/(?:www\.)?[-a-zA-Z0-9@:%._+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}[-a-zA-Z0-9()@:%_+.~#?&/=]*)' 

client = tweepy.Client(bearer_token = bearer_token3)

twitter_data_list = []
for i in range(10):
    start = i * 100
    end = start + 100
    df_tweet_subset = df_tweets_pro[start: end]
    acc_type_dict = dict(zip(df_tweet_subset["id"].astype(str), df_tweet_subset["account_type"]))
    each_id_list = list(df_tweet_subset["id"])
    print(str(start) + " : " + str(end)) 
    twitter_data = get_features_dictionary(each_id_list, acc_type_dict, client)
    tweets_df = pd.DataFrame(twitter_data)
    twitter_data_list.append(tweets_df)

combined_twitter_data = pd.concat(twitter_data_list)

combined_twitter_data["tweets"] = ""

combined_twitter_data.to_csv(f'combined/combined_twitter_data32.csv')


1700 : 1740


# START

In [296]:
df_celeb["account_type"] = "human"
df_celeb

,twitter,domain,name,type,account_type
0,katyperry,katyperry.com,KATY PERRY,celebrity,human
1,justinbieber,smarturl.it,Justin Bieber,celebrity,human
2,taylorswift13,grmypro.co,Taylor Swift,celebrity,human
3,rihanna,rihannanow.com,Rihanna,celebrity,human
4,ladygaga,NaN,The Countess,celebrity,human
...,...,...,...,...,...
991,Zedd,smarturl.it,Zedd,celebrity,human
992,SleepintheGardn,katuriankaturiankaturian.tumblr.com,Troian,celebrity,human
993,therealjuicyj,thejuicyj.com,juicy j,celebrity,human
994,manugavassi,manugavassi.com.br,Manu Gavassi,celebrity,human


In [383]:
# name, username, created_at, protected, verified, location, description, 
# public_metrics (followers_count, following_count, tweets_count)
# id, tweets, account_type
# CHECK
# favourites_count, geo_enabled, statuses_count
# average_tweets_per_day, account_age_days

client = tweepy.Client(bearer_token = bearer_token3)

def get_user_dictionary(screen_name, client):
    twitter_data = {'id':[], 'name':[], 'username':[], 'created_at':[], 
                    'protected':[], 'verified':[], 'location':[], 
                    'description':[], 'followers_count':[], 'following_count':[],
                    'tweet_count':[]}

    
    user = client.get_user(username=screen_name,
                                user_fields=["created_at",
                                             "description",
                                             "entities",
                                             "id",
                                             "location",
                                             "name",
                                             "profile_image_url",
                                             "public_metrics",
                                             "url",
                                             "username",
                                             "verified", 
                                             "protected"])
    user_dict = user.data
    
    if user_dict == None:
        return None
    
    if 'id' in user_dict:
        twitter_data['id'] = user_dict['id']
    else:
        twitter_data['id'] = ''
    
    if 'name' in user_dict:
        twitter_data['name'] = user_dict['name']
    else:
        twitter_data['name'] = ''

    if 'username' in user_dict:
        twitter_data['username'] = user_dict['username']
    else:
        twitter_data['username'] = ''

    if 'created_at' in user_dict:
        twitter_data['created_at'] = user_dict['created_at']
    else:
        twitter_data['created_at'] = ''

    if 'protected' in user_dict:
        twitter_data['protected'] = user_dict['protected']
    else:
        twitter_data['protected'] = ''

    if 'verified' in user_dict:
        twitter_data['verified'] = user_dict['verified']
    else:
        twitter_data['verified'] = ''

    if 'location' in user_dict:
        twitter_data['location'] = user_dict['location']
    else:
        twitter_data['location'] = ''

    if 'description' in user_dict:
        twitter_data['description'] = user_dict['description']
    else:
        twitter_data['description'] = ''

    if 'public_metrics' in user_dict:
        each_pub_metric = user_dict['public_metrics']

        if 'followers_count' in each_pub_metric:
            twitter_data['followers_count'] = each_pub_metric['followers_count']
        else:
            twitter_data['followers_count'] = ''

        if 'following_count' in each_pub_metric:
            twitter_data['following_count'] = each_pub_metric['following_count']
        else:
            twitter_data['following_count'] = ''

        if 'tweet_count' in each_pub_metric:
            twitter_data['tweet_count'] = each_pub_metric['tweet_count']
        else:
            twitter_data['tweet_count'] = ''
    else:
        twitter_data['followers_count'] = ''
        twitter_data['following_count'] = ''
        twitter_data['tweet_count'] = ''
    return twitter_data

In [317]:
bearer_token1 = 'AAAAAAAAAAAAAAAAAAAAADuChwEAAAAAyd5NyoPPZfk%2FiBwmc2mC9me33RA%3DTFH93ScdBzcU6OHVLLsTDHKLW599NhhPoEBTPi0KFWdAEbmFth'
bearer_token2 = 'AAAAAAAAAAAAAAAAAAAAAAK%2FhwEAAAAAtHE4qeLy3hhb660R143d7IOccaE%3D8qaWBsRfyKMigZj2jBVVFNifngriGAkuwJv74wOOs05m6XNReW'
bearer_token3 = 'AAAAAAAAAAAAAAAAAAAAAOjAhwEAAAAAzL7KuhOWC8yybNEnpbXPfhmCpMA%3Ds3R17rqHTA6rqR2kDhis6mrjFWYth3VgLgUu6w354BlwY2hGH8'

bearer_tokens = [bearer_token3, bearer_token1, bearer_token2]

In [384]:
start_index = 9
pattern = r'(https?:\/\/(?:www\.)?[-a-zA-Z0-9@:%._+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}[-a-zA-Z0-9()@:%_+.~#?&/=]*)' 

client = tweepy.Client(bearer_token = bearer_token3)

twitter_data_list = []
for i in range(100):
    twitter_data = get_user_dictionary(df_celeb["twitter"][i], client)
    if twitter_data == None:
        continue
    twitter_df = pd.DataFrame([twitter_data])
    twitter_data_list.append(twitter_df)
    
combined_twitter_data = pd.concat(twitter_data_list)

combined_twitter_data["tweets"] = ""

combined_twitter_data.to_csv(f'combined_celeb/celeb_data5.csv')


TooManyRequests: 429 Too Many Requests
Too Many Requests

In [382]:
df = pd.DataFrame({'A': ['bat', '', 'bait'],
                   'B': ['abc', 'bar', 'xyz']})

In [381]:
df['A'] = df['A'].replace([], '')
df

,A,B
0,bat,abc
1,[],bar
2,bait,xyz


In [ ]:
# combined_twitter_data["tweets"] = ""

# combined_twitter_data.to_csv(f'combined/celeb_data1.csv')

# END

#### ETL

In [276]:
## run from here in the future

directory = os.getcwd() + "\combined"
file_list = os.listdir(directory)

df_combined_list = []
for each_file in file_list:
    file = pd.read_csv(f'combined\{each_file}')
    file = file.drop(columns = ['Unnamed: 0'])
    df_combined_list.append(file)

In [277]:
final_combined = pd.concat(df_combined_list)

In [278]:
final_combined = final_combined.reset_index(inplace = False, drop = True)

In [279]:
final_combined['id'] = final_combined['id'].astype(str)

In [375]:
## run from here in the future

directory = os.getcwd() + "\combined_celeb"
file_list = os.listdir(directory)

df_celeb_list = []
for each_file in file_list:
    file = pd.read_csv(f'combined_celeb\{each_file}')
    file = file.drop(columns = ['Unnamed: 0'])
    df_celeb_list.append(file)

In [376]:
celeb_combined = pd.concat(df_celeb_list)

In [377]:
celeb_combined = celeb_combined.reset_index(inplace = False, drop = True)

In [379]:
celeb_combined

,id,name,username,created_at,protected,verified,location,description,followers_count,following_count,tweet_count,tweets
0,,KATY PERRY,katyperry,2009-02-20 23:45:56+00:00,False,True,NaN,LOVE is the key that unlocks every door🗝️♥️,108877952,245,11778,NaN
1,,Justin Bieber,justinbieber,2009-03-28 16:41:22+00:00,False,True,The 6,JUSTICE the album out now,113824591,281734,31332,NaN
2,,Taylor Swift,taylorswift13,2008-12-06 10:10:54+00:00,False,True,NaN,"I'm the problem, it's me",91736159,0,742,NaN
3,,Rihanna,rihanna,2009-10-02 21:37:33+00:00,False,True,NaN,NaN,107129513,982,10656,NaN
4,,Lady Gaga,ladygaga,2008-03-26 22:37:48+00:00,False,True,NaN,THE CHROMATICA BALL ⚔️💓 | Hold My Hand ✈️ Out ...,84952042,117032,9954,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
395,,Felipe Neto 🦉,felipeneto,2008-03-12 00:17:35+00:00,False,True,Rio de Janeiro,Cover da Kelly Key /// Contato Comercial: feli...,15553355,803,760,NaN
396,,Polo Polo,polo_polo,2008-08-21 17:27:53+00:00,False,True,México,"Palabras más, palabras menos, simplemente algu...",4758258,38,6227,NaN
397,,GloriaTrevi,GloriaTrevi,2009-10-29 17:34:24+00:00,False,True,NaN,#IslaDivina #ElSeEquivoco,5067054,1500,56698,NaN
398,,Vijay Mallya,TheVijayMallya,2010-03-31 07:48:39+00:00,False,True,India,"Former two term Member of Parliament, Chairman...",5774273,252,12207,NaN


#### Features

In [280]:
final_combined_others = final_combined.drop_duplicates(subset=['id'], keep='first')
final_combined_others = final_combined_others.drop(columns = ['is_quoted_retweets'])
final_combined_others['id'].nunique()

35698

In [281]:
def convert_TF(variable):
    if variable == 'True':
        return True
    if variable == 'False':
        return False
    if variable == '0.0':
        return False
    if variable == '1.0':
        return True
    return variable

boolean_var = ["verified", "protected"]

for var in boolean_var:
    final_combined_others[var] = final_combined_others[var].apply(convert_TF)

In [282]:
others_dataset = final_combined_others.drop(columns = ['tweets'])
others_dataset.to_csv('others_dataset.csv', index = False)
others_dataset.to_csv('../2. Feature Engineering/others_dataset.csv', index = False)

In [283]:
others_dataset.shape

(35698, 13)

#### Tweets

In [14]:
final_combined = final_combined[~final_combined['tweets'].str.startswith("RT @")]
final_combined = final_combined.drop(columns = ['protected', 'is_quoted_retweets'])

In [16]:
tweet_dataset = final_combined[['id', 'tweets', 'account_type']]
tweet_dataset.to_csv('tweet_dataset.csv', index = False)

In [17]:
tweet_dataset.shape

(88132, 3)

In [18]:
final_combined.to_csv('final_twitter_dataset.csv', index = False)